In [1]:
# Get http://geneontology.org/ontology/go-basic.obo
from goatools.base import download_go_basic_obo
obo_fname = download_go_basic_obo()

#from goatools.obo_parser import GODag
#obodag = GODag("C:/Users/User/Documents/KBI_DATA/go-basic.obo")'''

#from goatools import *
#obodag=obo_parser.GODag("C:/Users/User/Documents/KBI_DATA/go-basic.obo")

  EXISTS: go-basic.obo


In [2]:
# Get ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz
from goatools.base import download_ncbi_associations
fin_gene2go = download_ncbi_associations()

  EXISTS: gene2go


In [3]:
from goatools.obo_parser import GODag

obodag = GODag("go-basic.obo")

go-basic.obo: fmt(1.2) rel(2019-07-01) 47,413 GO Terms


In [4]:
from __future__ import print_function
from goatools.anno.genetogo_reader import Gene2GoReader

# Read NCBI's gene2go. Store annotations in a list of namedtuples
objanno = Gene2GoReader(fin_gene2go, taxids=[10090])

# Get namespace2association where:
#    namespace is:
#        BP: biological_process               
#        MF: molecular_function
#        CC: cellular_component
#    assocation is a dict:
#        key: NCBI GeneID
#        value: A set of GO IDs associated with that gene
ns2assoc = objanno.get_ns2assc()

for nspc, id2gos in ns2assoc.items():
    print("{NS} {N:,} annotated mouse genes".format(NS=nspc, N=len(id2gos)))

HMS:0:00:05.873129 366,685 annotations READ: gene2go 
1 taxids stored: 10090
MF 16,801 annotated mouse genes
CC 18,979 annotated mouse genes
BP 17,808 annotated mouse genes


In [5]:
from goatools.test_data.genes_NCBI_10090_ProteinCoding import GENEID2NT as GeneID2nt_mus

In [6]:
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS

goeaobj = GOEnrichmentStudyNS(
        GeneID2nt_mus.keys(), # List of mouse protein-coding genes
        ns2assoc, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest correction method


Load BP Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 60% 16,810 of 28,212 population items found in association

Load CC Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 65% 18,323 of 28,212 population items found in association

Load MF Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 58% 16,419 of 28,212 population items found in association


In [7]:
# Data will be stored in this variable
import os
geneid2symbol = {}
# Get xlsx filename where data is stored
ROOT = os.path.dirname(os.getcwd()) # go up 1 level from current working directory
din_xlsx = os.path.join(ROOT, "C:/Users/User/Documents/KBI_DATA/nbt.3102-S4_GeneIDs.xlsx")
# Read data
if os.path.isfile(din_xlsx):  
    import xlrd
    book = xlrd.open_workbook(din_xlsx)
    pg = book.sheet_by_index(0)
    for r in range(pg.nrows):
        symbol, geneid, pval = [pg.cell_value(r, c) for c in range(pg.ncols)]
        if geneid:
            geneid2symbol[int(geneid)] = symbol
    print('{N} genes READ: {XLSX}'.format(N=len(geneid2symbol), XLSX=din_xlsx))
else:
    raise RuntimeError('FILE NOT FOUND: {XLSX}'.format(XLSX=din_xlsx))

400 genes READ: C:/Users/User/Documents/KBI_DATA/nbt.3102-S4_GeneIDs.xlsx


In [8]:
# 'p_' means "pvalue". 'fdr_bh' is the multipletest method we are currently using.
geneids_study = geneid2symbol.keys()
goea_results_all = goeaobj.run_study(geneids_study)
goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]


Run BP Gene Ontology Analysis: current study set of 400 IDs ...
 94%    358 of    382 study items found in association
 96%    382 of    400 study items found in population(28212)
Calculating 12,237 uncorrected p-values using fisher_scipy_stats
  12,237 GO terms are associated with 16,810 of 28,212 population items
   2,083 GO terms are associated with    358 of    400 study items
  METHOD fdr_bh:
      72 GO terms found significant (< 0.05=alpha) ( 70 enriched +   2 purified): statsmodels fdr_bh
     231 study items associated with significant GO IDs (enriched)
       4 study items associated with significant GO IDs (purified)

Run CC Gene Ontology Analysis: current study set of 400 IDs ...
 98%    376 of    382 study items found in association
 96%    382 of    400 study items found in population(28212)
Calculating 1,735 uncorrected p-values using fisher_scipy_stats
   1,735 GO terms are associated with 18,323 of 28,212 population items
     446 GO terms are associated with    376 o

In [9]:
goeaobj.wr_xlsx("nbt3102.xlsx", goea_results_sig)
goeaobj.wr_txt("nbt3102.txt", goea_results_sig)

    217 items WROTE: nbt3102.xlsx
    217 GOEA results for   375 study items. WROTE: nbt3102.txt


In [10]:
from goatools.godag_plot import plot_gos, plot_results, plot_goid2goobj

plot_results("C:/Users/User/Documents/KBI_DATA/nbt3102_{NS}.png", goea_results_sig)

   72 usr 500 GOs  WROTE: C:/Users/User/Documents/KBI_DATA/nbt3102_BP.png
   90 usr 182 GOs  WROTE: C:/Users/User/Documents/KBI_DATA/nbt3102_CC.png
   55 usr 150 GOs  WROTE: C:/Users/User/Documents/KBI_DATA/nbt3102_MF.png


In [11]:
# Plot subset starting from these significant GO terms
goid_subset = [
    'GO:0003723', # MF D04 RNA binding (32 genes)
    'GO:0044822', # MF D05 poly(A) RNA binding (86 genes)
    'GO:0003729', # MF D06 mRNA binding (11 genes)
    'GO:0019843', # MF D05 rRNA binding (6 genes)
    'GO:0003746', # MF D06 translation elongation factor activity (5 genes)
]
plot_gos("C:/Users/User/Documents/KBI_DATA/nbt3102_MF_RNA_genecnt.png", 
    goid_subset, # Source GO ids
    obodag, 
    goea_results=goea_results_all) # Use pvals for coloring

    5 usr  13 GOs  WROTE: C:/Users/User/Documents/KBI_DATA/nbt3102_MF_RNA_genecnt.png
